### Imports

In [24]:
%reload_ext autoreload
%autoreload 2
# import snscrape.modules.twitter as sntwitter
import pandas as pd
import tqdm
import yaml
#
import json
import time
import datetime
import tqdm 
import os
# from datetime import datetime, timezone, timedelta
import re
from twarc.client2 import Twarc2
from twarc.expansions import ensure_flattened, flatten
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
# import twarc_csv
from twarc_csv import CSVConverter
from v2_csv_converter import DataFrameConverter
from rtweet_conversion import extra_cols, torename



LIGHT conversion columns

In [4]:
dropcols=['display_text_width',
    'urls_url', # Normalement c'est la liste des URLS mentionnées, tronqué par rtweets. urls_expanded_url contient la 1ère (?) de ces URLS
    'urls_t.co', 
    'geo_coords', # Vide dans Rtweets, on conserve "coords_coords"
    'favorite_count', # Même chose que les likes, existe en double dans rtweets
    'profile_banner_url', 
    'profile_background_url',
    'media_expanded_url', # C'est le lien vers l'image dans le status, on peut le reconstruire si besoin. Dans la v2 pas d'expanded_url.
    'media_t.co',
    # Ext_media : Dans la v1, renvoie une liste si plusieurs medias (4 photos max)
    # On conserve ext_media_url
    'ext_media_expanded_url', 
    'ext_media_t.co',
    'profile_url', # N'existe pas V2
    'profile_expanded_url',
    'account_lang', # Vide
    'reply_to_screen_name', # Beaucoup de données manquantes, TODO les ré-imputer depuis le JSON ?
    'profile_image_url', # Inutile
    'media_url' # Redondant avec ext_media_url
    ]

In [25]:
dropcols=['display_text_width',
    'urls_url', # Normalement c'est la liste des URLS mentionnées, tronqué par rtweets. urls_expanded_url contient la 1ère (?) de ces URLS
    'urls_t.co', 
    'geo_coords', # Vide dans Rtweets, on conserve "coords_coords"
    'favorite_count', # Même chose que les likes, existe en double dans rtweets
    'profile_banner_url', 
    'profile_background_url',
    'media_expanded_url', # C'est le lien vers l'image dans le status, on peut le reconstruire si besoin. Dans la v2 pas d'expanded_url.
    'media_t.co',
    # Ext_media : Dans la v1, renvoie une liste si plusieurs medias (4 photos max)
    # On conserve ext_media_url
    'ext_media_expanded_url', 
    'ext_media_t.co',
    'profile_url', # N'existe pas V2
    'profile_expanded_url',
    'account_lang', # Vide
    'reply_to_screen_name', # Beaucoup de données manquantes, TODO les ré-imputer depuis le JSON ?
    'profile_image_url', # Inutile
    'media_url', # Redondant avec ext_media_url
    # many retweet fields are'nt useful since they inherit everything from the original tweet
    # 'retweet_location',
    # 'retweet_description',
    'retweet_verified',
    'retweet_text',
    'retweet_source',
    'retweet_retweet_count',
    'retweet_favorite_count']

In [26]:
from v2_csv_converter import DEFAULT_TWEET_COLUMNS
# from rtweet_conversion import dropcols
tokeep=[k for k,v in torename.items() if v not in dropcols]+extra_cols+['context_annotations','entities.annotations', 'geo.id']
print('tokeep columns : %d'%len(tokeep))
print('default columns : %d'%(len(DEFAULT_TWEET_COLUMNS)+len(extra_cols)+len(['context_annotations','entities.annotations', 'geo.id'])))
print(tokeep)


tokeep columns : 77
default columns : 112
['id', 'referenced_tweets.replied_to.id', 'referenced_tweets.retweeted.id', 'in_reply_to_user_id', 'referenced_tweets.quoted.id', 'author_id', 'retweeted_user_id', 'quoted_user_id', 'created_at', 'text', 'lang', 'source', 'public_metrics.like_count', 'public_metrics.quote_count', 'public_metrics.reply_count', 'public_metrics.retweet_count', 'author.created_at', 'author.username', 'author.name', 'author.description', 'author.location', 'author.pinned_tweet_id', 'author.protected', 'author.public_metrics.followers_count', 'author.public_metrics.following_count', 'author.public_metrics.listed_count', 'author.public_metrics.tweet_count', 'author.url', 'author.verified', 'geo.coordinates.coordinates', 'geo.country', 'geo.country_code', 'geo.full_name', 'geo.geo.bbox', 'geo.name', 'geo.place_id', 'geo.place_type', 'entities.cashtags', 'is_retweet', 'is_quote', 'retweet_location', 'quoted_text', 'retweet_description', 'quoted_screen_name', 'retweet_so

### LIGHT CONVERSION

JSON Regular

In [14]:
# We can also use :
# [obj for obj in os.scandir("dir") if obj.is_dir()] # obj.is_file(), obj.name
# Process multiple json files to csv, Light version
data=pd.DataFrame()
# Merge JSON for data collection that was split ('query1.json', 'query2.json'...) 
for f in os.scandir('newdatacollection')  :
    if 'Htags' in f.name :
        converter_L = DataFrameConverter(
        input_data_type="tweets",
        json_encode_all=False,
        json_encode_text=False,
        json_encode_lists=True,### TRUE ?
        inline_referenced_tweets=False,
        merge_retweets=True,
        allow_duplicates=False,
        extra_input_columns=','.join(extra_cols),
        output_columns=','.join(tokeep)
        )

        with open(f, "r") as infile:
            with open("newdatacollection/"+ f.name.rsplit('.', 1)[0] + '.csv', "w") as outfile:
                converter = CSVConverter(infile, outfile, converter=converter_L)
                converter.process()

  0%|          | Processed 0.00/482M of input file [00:00<?, ?B/s]

Same process if you have jsonl files (1 json object per line)

In [30]:
# Returns all filenames matching regex in dir and subdirs
import glob
from pathlib import Path
def get_files_w_extension(base_dir, extension):
    return glob.iglob(rf"{base_dir}\**\*.{extension}", recursive=True)
# Process multiple JSON files to CSV  LIGHT
data=pd.DataFrame()
i=0
for f in get_files_w_extension("MOBILISE_backup\\", "jsonl") :
    converter_L = DataFrameConverter(
    input_data_type="tweets",
    json_encode_all=False,
    json_encode_text=False,
    json_encode_lists=True,
    inline_referenced_tweets=False,
    merge_retweets=True,
    allow_duplicates=False,
    extra_input_columns=','.join(extra_cols),
    output_columns=','.join(tokeep)
)
    with open(f, "r") as infile:
        o_name = infile.name.rsplit('.', 1)[0].rsplit('\\',1)[-1]+'.csv'
        print('Converting file to '+o_name+' ...')
        with open("export_folder/"+o_name, "w") as outfile:
            converter = CSVConverter(infile, outfile, converter=converter_L)
            converter.process()
    i+=1

Converting file to 1608_2008_queries_tweets_inline_light.csv ...


  0%|          | Processed 0.00/2.16G of input file [00:00<?, ?B/s]

Converting file to 0820-0920_tweets_inline_light.csv ...


  0%|          | Processed 0.00/5.77G of input file [00:00<?, ?B/s]

Converting file to 0920_1030_tweets_inline_light.csv ...


...

  0%|          | Processed 0.00/1.09G of input file [00:00<?, ?B/s]

Converting file to hashtags2_tweets_inline_light.csv ...


  0%|          | Processed 0.00/740M of input file [00:00<?, ?B/s]

Converting file to hashtags_trending_short_light.csv ...


  0%|          | Processed 0.00/388M of input file [00:00<?, ?B/s]

## MERGING CSVS
Early batches of data were collected in the format 'data/query1/keywords0_to_1023.csv', 'data/query1/keywords1024_to_1300.csv'...   
While we will eventually merge the entire data, we want to keep one file per query. To answer questions such as, how many tweets did each query yield ?

In [13]:
to_merge = [f'data/newdata/query{n+1}' for n in range (5)]
for d in to_merge :
    data=pd.DataFrame()
    for f in os.scandir(d) :
        print(f)
        data=pd.concat([data, pd.read_csv(f)])
        data['id']=data['id'].astype(str)
        data.drop_duplicates(subset='id', inplace=True, keep='first')
    print('concatenating files to : '+d+'.csv')
    data.to_csv(d+'.csv')

<DirEntry '2020_0601_0808_g1q1.csv'>


<ipython-input-13-6cadaa39729c>:6: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.concat([data, pd.read_csv(f)])


<DirEntry '2020_0601_0808_g1q2.csv'>
concatenating files to : newdatacollection2/query1.csv
<DirEntry '2020_0601_0808_g2q1.csv'>


<ipython-input-13-6cadaa39729c>:6: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.concat([data, pd.read_csv(f)])


<DirEntry '2020_0601_0808_g2q2.csv'>


...

concatenating files to : newdatacollection2/query4.csv
<DirEntry '2020_0601_0701_g5q1.csv'>
<DirEntry '2020_0701_0715_g5q1.csv'>


<ipython-input-13-6cadaa39729c>:6: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.concat([data, pd.read_csv(f)])


<DirEntry '2020_0715_0731_g5q1.csv'>


<ipython-input-13-6cadaa39729c>:6: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.concat([data, pd.read_csv(f)])


<DirEntry '2020_0801_0808_g5q1.csv'>
concatenating files to : newdatacollection2/query5.csv


In [29]:
to_merge = [f'data/newdata/hashtags1_top100']
for d in to_merge :
    data=pd.DataFrame()
    for f in os.scandir(d) :
        print(f)
        data=pd.concat([data, pd.read_csv(f)])
        data['id']=data['id'].astype(str)
        data.drop_duplicates(subset='id', inplace=True, keep='first')
    print('concatenating files to : '+d+'.csv')
    data.to_csv('data/newdata/new_hashtags/hashtags_top100'+'.csv')

<DirEntry 'hashtags1_top100_a.csv'>


C:\Users\GGB\AppData\Local\Temp\ipykernel_11176\2500193842.py:6: DtypeWarning: Columns (30,38,75) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.concat([data, pd.read_csv(f)])


<DirEntry 'hashtags1_top100_b.csv'>


C:\Users\GGB\AppData\Local\Temp\ipykernel_11176\2500193842.py:6: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.concat([data, pd.read_csv(f)])


concatenating files to : data/newdata/hashtags1_top100.csv


Merge CSVs from subdirectories

In [33]:
# Merge multiple csv from subdirectories 
# such as ('subdir/keywords0_to_1023.csv', 'subdir/keywords1024_to_1300.csv') => subdir.csv
subdirs = [obj for obj in os.scandir("newexport") if obj.is_dir()] #may use obj.name
for s in subdirs :
    df = pd.DataFrame()
    for f in [file for file in os.scandir(s.path) if '.csv' in file.name] :
        df = pd.concat([df, pd.read_csv(f)])
        df = df.drop_duplicates(subset='id', keep='first')
    df.to_csv(s.path+'.csv')

C:\Users\GGB\AppData\Local\Temp\ipykernel_19476\3557953265.py:7: DtypeWarning: Columns (19,20,21,22,31,33,34,35,41,56,58) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat([df, pd.read_csv(f)])
C:\Users\GGB\AppData\Local\Temp\ipykernel_19476\3557953265.py:7: DtypeWarning: Columns (20,21,58,59,60) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat([df, pd.read_csv(f)])
C:\Users\GGB\AppData\Local\Temp\ipykernel_19476\3557953265.py:7: DtypeWarning: Columns (31,33,34,35,59,60) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat([df, pd.read_csv(f)])
C:\Users\GGB\AppData\Local\Temp\ipykernel_19476\3557953265.py:7: DtypeWarning: Columns (19,20,21,31,33,34,35,41,56,58,59,60) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat([df, pd.read_csv(f)])
C:\Users\GGB\AppData\Local\Temp\ipykernel_19476\3557953265.py:7: DtypeWarning: Columns (20